In [1]:
import catboost as cb
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
labels = pd.read_csv('labels_training_set.csv')

In [3]:
processed = pd.read_csv('features_05.csv')

In [4]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [5]:
processed.head()

,person,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,...,modelos_distintos,busquedas,clicks_en_ads,checkouts,entradas_desde_motor,listar_productos,listar_marca,tamanio_pantalla,entro_direct,uso_computer
0,0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,...,0,0,0,3,0,1,0,2073600,False,True
1,00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,...,36,0,15,2,0,0,25,786432,True,True
2,00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,...,3,0,1,0,0,0,5,230400,False,False
3,000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,...,26,0,1,6,1,14,24,786432,True,True
4,000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,...,1,9,1,1,1,1,0,230400,False,False


In [7]:
features = processed.set_index(processed.columns[0]).drop(['primer_visita','ultima_visita'],axis=1)

In [8]:
dataset = labels.set_index('person').join(features,how='left')

In [9]:
dataset.head()

,label,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,...,modelos_distintos,busquedas,clicks_en_ads,checkouts,entradas_desde_motor,listar_productos,listar_marca,tamanio_pantalla,entro_direct,uso_computer
person,,,,,,,,,,,,,,,,,,,,,
0566e9c1,0,viewed product,iPhone 5s,Smartphone,1,17,23,68,9,0,...,12,0,6,1,1,15,3,181760,False,False
6ec7ee77,0,visited site,Ninguno,Smartphone,0,1,0,2,1,3,...,0,0,1,0,0,0,0,230400,False,False
abe7a2fb,0,viewed product,iPhone 6,Smartphone,0,22,31,96,109,2,...,7,6,9,1,4,9,14,230400,False,False
34728364,0,viewed product,Lenovo Vibe K5,Smartphone,0,4,24,37,9,4,...,11,0,0,0,1,3,5,230400,False,False
87ed62de,0,viewed product,Samsung Galaxy J7,Computer,0,1,9,17,1,12,...,1,0,5,2,0,0,0,995328,False,True


In [10]:
for x in dataset.columns:
    if dataset[x].dtype == 'object':
        dataset[x]=dataset[x].astype('category')

In [11]:
training_features = dataset.drop(['label'],axis=1)
training_labels = dataset['label']

In [12]:
xtrain,xtest,ytrain,ytest = train_test_split(training_features,training_labels)

In [13]:
catBoost = cb.CatBoostClassifier(iterations=200,
                                 depth=8,
                                 l2_leaf_reg=50,
                                 random_strength=0.1,
                                 bagging_temperature=0,
                                 border_count=128)

In [14]:
cat = []
for index,v in enumerate(xtrain.columns):
    if xtrain[v].dtype.name == 'category':
        cat.append(index)


In [15]:
catBoost.fit(xtrain,ytrain,verbose=0,cat_features=cat)

In [16]:
roc_auc_score(ytest,catBoost.predict_proba(xtest)[:,1])

0.8501255317893688

In [17]:
catBoost.fit(training_features,training_labels,verbose=0,cat_features=cat)

In [18]:
prediction_features = kaggle.set_index('person').join(features,how='left')

In [19]:
pred = catBoost.predict_proba(prediction_features)[:,1]
submit = kaggle.set_index('person')
submit['label'] = pred

In [20]:
submit.head()

,label
person,
4886f805,0.007907
0297fc1e,0.060722
2d681dd8,0.016244
cccea85e,0.052161
4c8a8b93,0.024271


In [21]:
submit.to_csv('submit.csv')